## OPENAI Frontier LLM Project

build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where I am loading in the environment variables in your `.env` file and connect to OpenAI.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. 
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)


Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [12]:
# A function that creates a list of messages for the API call:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThe website is a personal portal for Ed Donner, who is a software engineer, co-founder, and CTO of Nebula.io. His focus is on creating and experimenting with large language models (LLMs) that assist in talent discovery and engagement. Ed expresses a passion for coding, DJing, and engaging with tech communities.\n\n### Notable Content\n- **About Ed:** Ed shares his background in AI and tech startups, mentioning his previous work with untapt, which was acquired in 2021. He emphasizes the impact of AI on personal development and talent management through Nebula.io.\n  \n- **Connect Four and Outsmart:** Features that involve competitive interactions among LLMs, but further details are not provided.\n\n### News and Announcements\n1. **LLM Workshop – Hands-on with Agents (January 23, 2025):** Resources released for participants of the workshop.\n2. **Welcome, SuperDataScientists! (December 21, 2024):** A greeting or announcement that likely pertains t

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website serves as a personal platform for Ed Donner, a software engineer and co-founder of Nebula.io, which focuses on utilizing AI to enhance talent discovery and engagement. Ed shares his passion for coding, experimenting with large language models (LLMs), and electronic music, alongside his experiences in the tech and AI sectors.

## Recent News and Announcements

- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources were made available.
- **December 21, 2024**: A welcoming note for SuperDataScientists was posted.
- **November 13, 2024**: Resources for "Mastering AI and LLM Engineering" were shared.
- **October 16, 2024**: Information on transitioning from Software Engineer to AI Data Scientist was provided, including resources.

The website encourages connections and engages visitors with its content on LLMs and AI applications in talent management.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [37]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN provides a wide range of news coverage, including breaking news, political developments, business updates, health information, entertainment, and sports. The site features dedicated sections for both domestic (U.S.) and international news, with topical areas such as the Ukraine-Russia War and the Israel-Hamas conflict. 

### Key Highlights:
- **Latest Updates**:
  - The White House has denied defying a judge’s order regarding Venezuelan deportations amid ongoing Trump administration policies.
  - Retailers have raised warnings about economic impacts attributed to Trump’s tariffs.
  - Reports on tragic events, including a storm system causing significant destruction and loss of life.

- **Politics**:
  - A CNN poll indicates a record low favorability for the Democratic Party.
  - Donald Trump and Vladimir Putin are scheduled to discuss potential peace talks.

- **Health and Science**:
  - Recent studies suggest gender-affirming hormone therapy may decrease depression in transgender adults.

- **Entertainment**:
  - Notable stories include Eva Longoria's star-studded birthday celebration and updates on major cultural events.

CNN also shares technology trends, environmental updates, and a range of lifestyle content, while encouraging users to provide feedback about their advertising experience.

In [38]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI safety and research company based in San Francisco, focused on creating reliable and beneficial AI systems with an emphasis on safety. The site highlights their latest AI model, **Claude 3.7 Sonnet**, which is described as the most intelligent model to date, featuring hybrid reasoning capabilities. The website also introduces **Claude Code**, a tool designed for coding tasks.

## Recent Announcements
- **Claude 3.7 Sonnet Release**: Announced as the most intelligent AI model, emphasizing enhanced reasoning abilities.
- **Claude Code Launch**: A new tool aimed at providing support for coding tasks.

The company's mission centers around AI safety, research, and transparent practices, with a commitment to responsible scaling, security, and compliance.